# 09wk-2: `model`의 입력파악, `model`의 사용연습

최규빈  
2024-11-09

<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/posts/09wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-xsRgsXpLEUNWClxLi0N9Mk&si=Lssxzw70RRT1adiJ>

In [1]:
!pip uninstall mp2024pkg -y 
!pip install git+https://github.com/guebin/mp2024pkg.git

Found existing installation: mp2024pkg 1.0
Uninstalling mp2024pkg-1.0:
  Successfully uninstalled mp2024pkg-1.0
  Cloning https://github.com/guebin/mp2024pkg.git to /tmp/pip-req-build-178x45er
  Running command git clone --filter=blob:none --quiet https://github.com/guebin/mp2024pkg.git /tmp/pip-req-build-178x45er
  Resolved https://github.com/guebin/mp2024pkg.git to commit 2adb5fc04e589895edd76ec5da87aa37bf0dd70b
  Preparing metadata (setup.py) ... done
  Created wheel for mp2024pkg: filename=mp2024pkg-1.0-py3-none-any.whl size=4588 sha256=84356f47d6566dcda64e0d93070d9a1d9b6d9c9f962769858a4d823b589fe40b
  Stored in directory: /tmp/pip-ephem-wheel-cache-n0_tsfan/wheels/83/81/b1/63c03ca869bb1a35a588dce54e83ea32c6c21e6af0bbeaa640
Successfully built mp2024pkg

# 2. Imports

In [2]:
import transformers
import datasets
import huggingface_hub
import torch
import torchvision
import pytorchvideo.data
import PIL
import tarfile
import mp2024pkg as mp

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

# 3. Model 입력파악

`-` 아래중 하나의 방법순으로..

-   방법1: `model.forward?` 에서 시그니처를 확인
-   방법2: `model.forward?` 에서 사용예제를 확인
-   방법3: 인터넷을 활용한 외부 자료 확인 (공식문서, 공식튜토리얼,
    신뢰할만한 블로그, ChatGPT등)
-   방법4: `model.forward??` 를 보고 모든 코드를 뜯어봄 \<— 하지마세요

`-` 모델의 입력이 어떤형태로 정리되어야 하는지 알아내는 확실한 방법은
없음

-   방법1,2,3 은 다른사람의 호의에 기대해야함.
-   방법4는 사실상 불가능

`# 예제1` – 텍스트분류

In [70]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

*모델의 기본정보(config)*

In [71]:
model1.config

-   max_position_embeddings: 512

*모델의 입력파악*

In [83]:
# model1.forward? 
# -- 결과가 너무 길어서 생략

*사용예시1 – 입력나열, loss O*

In [6]:
model1(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]]),
    labels = torch.tensor([0,0])
)

*사용예시2 – `**딕셔너리`, loss O*

In [7]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]]),
    labels = torch.tensor([0,0])
)
model1(**model1_input)

*사용예시3 – 입력나열, loss X*

In [8]:
model1(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]])
)

*사용예시4 – `**딕셔너리`, loss X*

In [9]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]])
)
model1(**model1_input)

*사용예시5 – 초간단, loss X*

In [10]:
model1(torch.tensor([[1,2,3,4], [2,3,4,5]]))

> 사용예시1~5에서 `model1()` 대신에 `model1.forward()`를 사용해도 된다.

`#`

`# 예제2` – 이미지분류

In [11]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

*모델의 기본정보(config)*

In [73]:
model2.config

-   image_size: 224

In [85]:
# mp.tab(model2)
# -- 결과가 너무 길어서 생략 

In [75]:
model2.config.num_channels

*모델의 입력파악*

In [86]:
# model2.forward?
# -- 결과가 너무 길어서 생략

In [15]:
torch.randn(2,3,64,64)

*사용예시1 – 입력나열, loss O*

In [16]:
torch.random.manual_seed(42)
model2(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,1])
)

*사용예시2 – `**딕셔너리`, loss O*

In [17]:
torch.random.manual_seed(42)
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,1])
)
model2(**model2_input)

*사용예시3 – 입력나열, loss X*

In [18]:
torch.random.manual_seed(42)
model2(
    pixel_values = torch.randn(2,3,224,224)
)

*사용예시4 – `**딕셔너리`, loss X*

In [19]:
torch.random.manual_seed(42)
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
)
model2(**model2_input)

*사용예시5 – 초간단, loss X*

In [20]:
torch.random.manual_seed(42)
model2(torch.randn(2,3,224,224))

`#`

`# 예제3` – 동영상분류

In [21]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

*모델의 기본정보(config)*

In [22]:
model3.config

-   image_size: 224
-   num_frames: 16

*모델의 입력파악*

In [87]:
# model3.forward?
# -- 결과가 너무 길어서 생략

*사용예시1 – 입력나열, loss O*

In [24]:
torch.random.manual_seed(42)
model3(
    pixel_values = torch.randn(4,16,3,224,224),
    labels = torch.tensor([0,1,0,1])
)

*사용예시2 – `**딕셔너리`, loss O*

In [25]:
torch.random.manual_seed(42)
model3_input = dict(
    pixel_values = torch.randn(4,16,3,224,224),
    labels = torch.tensor([0,1,0,1])
)
model3(**model3_input)

*사용예시3 – 입력나열, loss X*

In [26]:
torch.random.manual_seed(42)
model3(
    pixel_values = torch.randn(4,16,3,224,224)
)

*사용예시4 – `**딕셔너리`, loss X*

In [27]:
torch.random.manual_seed(42)
model3_input = dict(
    pixel_values = torch.randn(4,16,3,224,224),
)
model3(**model3_input)

*사용예시5 – 초간단, loss X*

In [28]:
torch.random.manual_seed(42)
model3(torch.randn(4,16,3,224,224))

`#`

# 4. Model 사용 연습

## A. 텍스트

In [29]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

`# 예제1` – imdb

In [30]:
imdb = datasets.load_dataset('imdb')

In [31]:
d = imdb['train'].select(range(3))
d

`(풀이1)`

*실패*

In [32]:
model1.forward(torch.tensor(tokenizer(d['text'])['input_ids']))

*원인분석*

In [33]:
mp.show_list(
    tokenizer(d['text'])['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 363
   - Value: [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 205

In [34]:
mp.show_list(
    tokenizer(d['text'], padding=True)['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 363
   - Value: [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 205

*성공*

In [33]:
model1(torch.tensor(tokenizer(d['text'], padding=True)['input_ids']))

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.

`(풀이2)`

In [34]:
model1(tokenizer(d['text'], padding=True, return_tensors="pt")['input_ids'])

`#`

`# 예제2` – emotion

In [35]:
emotion = datasets.load_dataset('emotion')

In [36]:
d = emotion['train'].select(range(3))
d

`(풀이)`

In [37]:
model1(torch.tensor(tokenizer(d['text'],padding=True)['input_ids']))

`#`

`# 예제3` – MBTI

In [38]:
# mbti1.csv 파일 다운로드 
!wget https://raw.githubusercontent.com/guebin/MP2024/refs/heads/main/posts/mbti_1.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
    - Avoid using `tokenizers` before the fork if possible
    - Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

--2024-11-15 19:18:39--  https://raw.githubusercontent.com/guebin/MP2024/refs/heads/main/posts/mbti_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62856486 (60M) [text/plain]
Saving to: ‘mbti_1.csv.1’

mbti_1.csv.1        100%[===================>]  59.94M   107MB/s    in 0.6s    

2024-11-15 19:18:44 (107 MB/s) - ‘mbti_1.csv.1’ saved [62856486/62856486]


In [39]:
d = datasets.Dataset.from_csv("mbti_1.csv").select(range(3))
d

Generating train split: 8675 examples [00:00, 16512.48 examples/s]

`(풀이1)`

*실패*

In [40]:
model1(torch.tensor(tokenizer(d['posts'],padding=True)['input_ids']))

Token indices sequence length is longer than the specified maximum sequence length for this model (2102 > 512). Running this sequence through the model will result in indexing errors

*원인분석*

In [41]:
mp.show_list(
    tokenizer(d['posts'],padding=True)['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 2102
   - Value: [101, 1005, 8299, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1053, 2015, 2595, 16257, 8545, 2509, 21638, 2860, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 4601, 1012, 2865, 1012, 10722, 14905, 20974, 1012, 4012, 1013, 10722, 14905, 20974, 1035, 1048, 14876, 26230, 2692, 2509, 9737, 27717, 19062, 2487, 3217, 9541, 2487, 1035, 3156, 1012, 16545, 2290, 1064, 1064, 1064, 4372, 22540, 1998, 20014, 3501, 5312, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1045, 2480, 2581, 2571, 2487, 2290, 2549, 2595, 2213, 2549, 2998, 13013, 2121, 2025, 2327, 2702, 3248, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 15384, 20952, 4371, 2487, 12870, 2278, 26418, 2015, 1064, 1064, 1064, 2054, 2038, 2042, 1996, 2087, 2166, 1011, 5278, 3325, 1999, 2115, 2166, 1029, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 7479, 10

In [42]:
mp.show_list(
    tokenizer(d['posts'],truncation=True)['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 512
   - Value: [101, 1005, 8299, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1053, 2015, 2595, 16257, 8545, 2509, 21638, 2860, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 4601, 1012, 2865, 1012, 10722, 14905, 20974, 1012, 4012, 1013, 10722, 14905, 20974, 1035, 1048, 14876, 26230, 2692, 2509, 9737, 27717, 19062, 2487, 3217, 9541, 2487, 1035, 3156, 1012, 16545, 2290, 1064, 1064, 1064, 4372, 22540, 1998, 20014, 3501, 5312, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1045, 2480, 2581, 2571, 2487, 2290, 2549, 2595, 2213, 2549, 2998, 13013, 2121, 2025, 2327, 2702, 3248, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 15384, 20952, 4371, 2487, 12870, 2278, 26418, 2015, 1064, 1064, 1064, 2054, 2038, 2042, 1996, 2087, 2166, 1011, 5278, 3325, 1999, 2115, 2166, 1029, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 7479, 101

*성공*

In [43]:
model1(torch.tensor(tokenizer(d['posts'],truncation=True)['input_ids']))
#model1(tokenizer(d['posts'],truncation=True,return_tensors="pt")['input_ids'])

`(풀이2)` *–모델설정변경 (퀴즈5, 모델의 프레임수를 4로 바꾸는 예제에서
사용한 테크닉)*

*distilbert/distilbert-base-uncased 설정값 부르기*

In [44]:
config = transformers.AutoConfig.from_pretrained(
    "distilbert/distilbert-base-uncased"
)
config

*설정값변경*

In [45]:
config.max_position_embeddings = 2200

*설정값으로 모델불러오기*

In [46]:
model1_large = transformers.AutoModelForSequenceClassification.from_config(
    config=config
)

*모델사용*

In [47]:
model1_large(torch.tensor(tokenizer(d['posts'],padding=True)['input_ids']))

In [48]:
model1_large(**tokenizer(d['posts'],padding=True,return_tensors="pt"))

`#`

`# 예제4` – sms_spam

In [49]:
sms_spam = datasets.load_dataset('sms_spam')['train'].train_test_split(test_size=0.2, seed=42)
sms_spam

In [50]:
d = sms_spam['train'].select(range(3))
d

`(풀이)`

In [51]:
model1(**tokenizer(d['sms'],padding=True,return_tensors="pt"))

`#`

## B. 이미지

In [52]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

`# 예제1` – food101

In [53]:
d = datasets.load_dataset("food101", split="train[:4]")
d

`(예비학습)` – `torchvision.transforms` 에서 제공하는 기능들은
배치처리가 가능한가?

In [54]:
to_tensor = torchvision.transforms.ToTensor()

In [55]:
to_tensor(d['image'][0])

In [56]:
to_tensor(d['image'])

`(풀이)`

In [58]:
compose = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224,224))
])

In [59]:
torch.stack(list(map(compose,d['image'])),axis=0).shape

In [60]:
model2.forward(
    torch.stack(list(map(compose,d['image'])),axis=0)
)

`#`

`# 예제2`

In [61]:
beans = datasets.load_dataset('beans')
d = beans['train'].select(range(4))
d

`(풀이)`

In [62]:
model2(torch.stack(list(map(compose,d['image'])),axis=0))

`#`

## C. 동영상

In [63]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

`# 예제1` – UCF101_subset

In [64]:
file_path = huggingface_hub.hf_hub_download(
    repo_id="sayakpaul/ucf101-subset",
    filename="UCF101_subset.tar.gz",
    repo_type="dataset"
)
# file_path는 다운로드한 압축파일이 존재하는 경로와 파일명이 string으로 저장되어있음.
with tarfile.open(file_path) as t:
     t.extractall("./data") # 여기에서 "."은 현재폴더라는 의미

In [65]:
mp.tree("./data")

└── UCF101_subset
    ├── test
    │   ├── ApplyEyeMakeup
    │   │   ├── UCF101
    │   │   ├── v_ApplyEyeMakeup_g03_c01.avi
    │   │   └── ...
    │   │   └── v_ApplyEyeMakeup_g23_c06.avi
    │   ├── ApplyLipstick
    │   │   ├── UCF101
    │   │   ├── v_ApplyLipstick_g14_c01.avi
    │   │   └── ...
    │   │   └── v_ApplyLipstick_g16_c04.avi
    │   └── ...
    │   └── BenchPress
    │       ├── UCF101
    │       ├── v_BenchPress_g05_c02.avi
    │       └── ...
    │       └── v_BenchPress_g25_c06.avi
    ├── train
    │   ├── ApplyEyeMakeup
    │   │   ├── UCF101
    │   │   ├── v_ApplyEyeMakeup_g02_c03.avi
    │   │   └── ...
    │   │   └── v_ApplyEyeMakeup_g25_c07.avi
    │   ├── ApplyLipstick
    │   │   ├── UCF101
    │   │   ├── v_ApplyLipstick_g01_c02.avi
    │   │   └── ...
    │   │   └── v_ApplyLipstick_g24_c05.avi
    │   └── ...
    │   └── BenchPress
    │       ├── UCF101
    │       ├── v_BenchPress_g01_c05.avi
    │       └── ...
    │       └── v_BenchPress_g24_c

In [66]:
video_path = "./data/UCF101_subset/test/BenchPress/v_BenchPress_g05_c02.avi"
video = pytorchvideo.data.encoded_video.EncodedVideo.from_path(video_path).get_clip(0, float('inf'))['video']
video.shape

`(풀이)`

In [67]:
model3(
    #video.permute(1,0,2,3)[:16,:,:224,:224].unsqueeze(0)
    #video.permute(1,0,2,3)[:16,:,:224,:224].reshape(1,16,3,224,224)
    torch.stack([video.permute(1,0,2,3)[:16,:,:224,:224]])
)